In [1]:
import pyomo.environ as pyEnv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import xlrd
from itertools import product
from sys import stdout as out
import sys

In [2]:
#Get data
file_path = 'data.xlsx'
df = pd.read_excel(file_path)
wb = xlrd.open_workbook(file_path) 
d = np.zeros((30,30))
p = np.zeros((30,30))
xc = np.zeros((4))
sheet_d = wb.sheet_by_index(0) 
sheet_p = wb.sheet_by_index(1)
for i in range(30):
    for j in range(30):
        d[i][j] = sheet_d.cell_value(i, j)
        p[i][j] = sheet_p.cell_value(i, j)
        

#UNCOMMENT FOR PART B
#for i in range(30):
#    for j in range(30):
#        if p[i][j] > 0.6:
#            d[i][j] = sys.maxsize


In [3]:
n = 30
#Model
model = pyEnv.ConcreteModel()
#Indexes for the cities
model.M = pyEnv.RangeSet(1,n)
model.N = pyEnv.RangeSet(1,n)
#Decision Variables xij
model.x = pyEnv.Var(model.N,model.M, within=pyEnv.Binary)

#Decision Variables yj
model.y = pyEnv.Var(model.M, within=pyEnv.Binary)

#dummy variable f
model.f = pyEnv.Var()

#Cost matrix dij
model.c = pyEnv.Param(model.N, model.M,initialize=lambda model, i, j: d[i-1][j-1])

In [4]:
model.O_f1 = pyEnv.Objective(expr= model.f  , sense=pyEnv.minimize)


In [5]:
 #Define Constraints
    
def rule_const0(model, N, M):
    return model.f >= model.x[N,M]*model.c[N,M]

model.const0 = pyEnv.Constraint(model.N,model.M,rule=rule_const0)
    
    
def rule_const1(model):
    return sum(model.y[j] for j in model.N) == 4

model.const1 = pyEnv.Constraint(rule=rule_const1)

def rule_const2(model, N):
    return sum(model.x[N, j] for j in model.N) == 1

model.const2 = pyEnv.Constraint(model.N,rule=rule_const2)

def rule_const3(model, N, M):
    return model.y[M] >= model.x[N,M]

model.const3 = pyEnv.Constraint(model.N,model.M,rule=rule_const3)



In [6]:
#model.pprint()

In [7]:
#Solves
solver = pyEnv.SolverFactory('cplex')
result = solver.solve(model)

#Prints the results
print(result)


Problem: 
- Name: tmp6v55arcg
  Lower bound: 50.5
  Upper bound: 50.5
  Number of objectives: 1
  Number of constraints: 1832
  Number of variables: 932
  Number of nonzeros: 4501
  Sense: minimize
Solver: 
- Status: ok
  User time: 0.44
  Termination condition: optimal
  Termination message: MIP - Integer optimal solution\x3a Objective = 5.0500000000e+01
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.4657094478607178
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [8]:
print("Center Villages: ", end="")
for i in range(1,31):
    if pyEnv.value(model.y[i]) == 1:
        print(i, end=" ")

for i in range(1,31):
    if pyEnv.value(model.y[i]) == 1:
        print("\n\nVillages that goes to ", i, ": ", end="")
    for j in range(1,31):
        if pyEnv.value(model.x[j,i]) == 1 and i != j:
            print(j, end=" ")

Center Villages: 15 19 24 30 

Villages that goes to  15 : 4 6 7 8 9 10 14 

Villages that goes to  19 : 5 11 12 16 20 23 25 26 28 

Villages that goes to  24 : 13 29 

Villages that goes to  30 : 1 2 3 17 18 21 22 27 